In [9]:
import os
from threading import Thread
import sys
import time
import cv2
import numpy
import argparse
import math
from glob import glob
import numpy as np
import json
import torch
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm

from task3_utils import match_pairs, MatchImageSizeTo,ocr

In [2]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
## Config
img_path = '/hub_data2/drone_2021sample/set05_drone01_30'
annotation_path = img_path + '_annotation'

## Data Load
img_list = glob(os.path.join(img_path, "*.png"))
img_list.sort()

img = []
for image in img_list:
    img.append(cv2.imread(image))
    
## Data prepare
img = np.stack(img, axis = 0)
print(img.shape)

(222, 1024, 1024, 3)


In [17]:
# %matplotlib inline


# img = Image.fromarray(img[0], 'RGB')

### OCR Inference

In [18]:
ocr_batch_size = 10

#### 의미없 (구색맞추기위함) ####
texts = [] 
text_idx = []
############################

torch.cuda.empty_cache()

## start ocr ##
results = []
Iters = math.ceil(img.shape[0]/ocr_batch_size)

with tqdm(total=Iters) as pbar:    
    for i in range(Iters):
        start = i * ocr_batch_size
        
        if i == Iters-1:
            end = img.shape[0]
        else:
            end = (i+1)*ocr_batch_size

        results.extend(ocr(img[start:end], start, np.array(range(0, img.shape[0])), texts, text_idx))
        pbar.update(1)
    
        break
torch.cuda.empty_cache()



  4%|▍         | 1/23 [00:04<01:36,  4.37s/it]


In [20]:
len(results)

10

### Metric

In [21]:
experiment_name = img_path.split('/')[-1]
save_path = os.path.join('result',experiment_name)
os.makedirs(save_path,exist_ok=True)

In [22]:
output_results = {}

experiment_name = img_path.split('/')[-1]
save_path = os.path.join('result',experiment_name)
os.makedirs(save_path,exist_ok=True)
print(save_path)

for idx, frame in enumerate(img_list):
    frame_name = frame.split('/')[-1].rstrip('.png') ### 'framexxx'
    # print(frame_name)
    
    output_results[frame_name] = {}
    output_results[frame_name]['png_path'] = frame ## png file path
    ### pred ###
    pred_info = results[idx]
    num_pred = len(pred_info)
    output_results[frame_name]['pred'] = {}
    
    for pred_idx in range(num_pred):
        output_results[frame_name]['pred'][pred_idx] = {}
        
        box_parms = pred_info[pred_idx]['boxes']
        text = pred_info[pred_idx]['text']
        confident = pred_info[pred_idx]['confident']
        
        x_min = int(np.min(np.array(box_parms)[:,0]))
        y_min = int(np.min(np.array(box_parms)[:,1]))
        x_max = int(np.max(np.array(box_parms)[:,0]))
        y_max = int(np.max(np.array(box_parms)[:,1]))
        
        ## 결과 정리
        output_results[frame_name]['pred'][pred_idx]['text'] = text
        output_results[frame_name]['pred'][pred_idx]['boxes'] = [x_min, y_min, x_max, y_max]
        output_results[frame_name]['pred'][pred_idx]['confident'] = confident
    
    ### GT ###
    output_results[frame_name]['gt'] = {}
    json_path = os.path.join(annotation_path, frame_name +'.json')
    
    if os.path.exists(json_path):
        with open(json_path, 'r') as f:
            json_data = json.load(f)
        
        gt_info = json_data[0]['annotations']
        num_gt = len(gt_info)
        
        for gt_idx in range(num_gt):
            output_results[frame_name]['gt'][gt_idx] = {}
            
            text = gt_info[gt_idx]['label']
            center_x = gt_info[gt_idx]['coordinates']['x']
            center_y = gt_info[gt_idx]['coordinates']['y']
            width = gt_info[gt_idx]['coordinates']['width']
            hegiht = gt_info[gt_idx]['coordinates']['height']
            
            x_min = int(center_x - width/2)
            y_min = int(center_y - hegiht/2)
            x_max = int(center_x + width/2)
            y_max = int(center_y + hegiht/2)
            
            ## 결과 정리
            output_results[frame_name]['gt'][gt_idx]['text'] = text
            output_results[frame_name]['gt'][gt_idx]['boxes'] = [x_min, y_min, x_max, y_max]

            
### Save result
# with open(os.path.join(save_path, 'result.json'), "w") as json_file:
#     json.dump(output_results, json_file)


result/set05_drone01_30


IndexError: list index out of range

### Visualization

In [25]:
experiment_name = 'set03_drone01_30_tmp'


save_path = os.path.join('result',experiment_name,'result.json')
with open(save_path, 'r') as f:
    output_json = json.load(f)

In [27]:
font_type = ImageFont.truetype("./arial.ttf", 18)

OSError: cannot open resource

In [18]:
frame = 'frame030'
frame_dict = output_json[frame]
png_path = frame_dict['png_path']

%matplotlib inline
plt.figure(figsize=(15,15))
img = cv2.imread(png_path)

print("--------- Pred (빨간색) ---------")
for pred in frame_dict['pred'].values():
    print(pred['text'])
    boxes = pred['boxes']
    cv2.rectangle(img,(boxes[0], boxes[1]),(boxes[2], boxes[3]), (255, 0 , 0), 2)
    cv2.putText(img[0], pred['text'], (boxes[0], boxes[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

print("--------- GT (초록색) ---------")
for gt in frame_dict['gt'].values():
    print(gt['text'])
    boxes = gt['boxes']
    
    img_PIL = Image.fromarray(img)
    draw = ImageDraw.Draw(img_PIL)
    draw.text((boxes[0], boxes[1]-10), str(gt['text']), font=font_type, fill=(0,255,0))
    
    cv2.rectangle(img,(boxes[0], boxes[1]),(boxes[2], boxes[3]), (0, 255, 0), 2)
    
plt.imshow(img)
# cv2.show()

--------- Pred (빨간색) ---------
72외고준비반
태위신애
떠
50대
--------- GT (초록색) ---------
302외고준비반


UnicodeEncodeError: 'latin-1' codec can't encode characters in position 3-7: ordinal not in range(256)

<Figure size 1080x1080 with 0 Axes>

In [8]:
img[0].shape

(1024, 3)

In [9]:
from task3 import Task3

In [10]:
task3 = Task3()


In [ ]:
# plt.figure(figsize=(15,15))
task3(img[30])